In [1]:
# Import packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
# Header to set the requests as a browser requests
headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

In [3]:
# URL of The amazon Review page
reviews_url = 'https://www.amazon.com/Legendary-Whitetails-Journeyman-Jacket-Tarmac/product-reviews/B013KW38RQ/'

In [21]:
# Define Page No
len_page = 300

### <font color="red">Functions</font>

In [22]:
# Extra Data as Html object from amazon Review page
def reviewsHtml(url, len_page):
    
    # Empty List define to store all pages html data
    soups = []
    
    # Loop for gather all 3000 reviews from 300 pages via range
    for page_no in range(1, len_page + 1):
        
        # parameter set as page no to the requests body
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }
        
        # Request make for each page
        response = requests.get(url, headers=headers)
        
        # Save Html object by using BeautifulSoup4 and lxml parser
        soup = BeautifulSoup(response.text, 'lxml')
        
        # Add single Html page data in master soups list
        soups.append(soup)
        
    return soups

In [23]:
# Grab Reviews name, description, date, stars, title from HTML
def getReviews(html_data):

    # Create Empty list to Hold all data
    data_dicts = []
    
    # Select all Reviews BOX html using css selector
    boxes = html_data.select('div[data-hook="review"]')
    
    # Iterate all Reviews BOX 
    for box in boxes:
        
        # Select Name using css selector and cleaning text using strip()
        # If Value is empty define value with 'N/A' for all.
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'

        try:
            stars = box.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            stars = 'N/A'   

        try:
            title = box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'

        try:
            # Convert date str to dd/mm/yyy format
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'

        try:
            description = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'

        # create Dictionary with al review data 
        data_dict = {
            'Name' : name,
            'Stars' : stars,
            'Title' : title,
            'Date' : date,
            'Description' : description
        }

        # Add Dictionary in master empty List
        data_dicts.append(data_dict)
    
    return data_dicts

### <font color="red">Data Process</font>

In [24]:
# Grab all HTML
html_datas = reviewsHtml(reviews_url, len_page)

In [25]:
# Empty List to Hold all reviews data
reviews = []

In [26]:
# Iterate all Html page 
for html_data in html_datas:
    
    # Grab review data
    review = getReviews(html_data)
    
    # add review data in reviews empty list
    reviews += review

In [27]:
# Create a dataframe with reviews Data
df_reviews = pd.DataFrame(reviews)

In [28]:
df_reviews

,Name,Stars,Title,Date,Description
0,Jeff,4.0,"4.0 out of 5 stars\nRugged, good look and nice...",02/10/2018,The short answer to if you should go down from...
1,Donny Smith,4.0,4.0 out of 5 stars\nNot bad,19/10/2023,"The Good: Looks great, it’s warm, seems to be ..."
2,William,5.0,5.0 out of 5 stars\nWish I could give it more ...,06/10/2023,It's not the most light weight of things but y...
3,TJ Michel,4.0,4.0 out of 5 stars\nHigh Quality,18/10/2023,The jacket is very nice. Was expecting it to ...
4,Jayce,5.0,5.0 out of 5 stars\nGlad I bought it,14/11/2023,It fits well. It's kind of big on me but I lik...
...,...,...,...,...,...
555,Señor Pavo,4.0,4.0 out of 5 stars\nWatch the price though,27/09/2023,This shirt is well made and I like the feeling...
556,Johnny Bradabadowski,5.0,5.0 out of 5 stars\nGreat!,01/11/2023,"Love the jacket! Looks good, feels good and fi..."
557,JEN BRADFORD,4.0,4.0 out of 5 stars\nGreat Jacket,02/11/2023,Would definitely be 5 stars but the right pock...
558,Troy Carothers,5.0,5.0 out of 5 stars\nLove it!,03/11/2023,I only wish I had bought it sooner. Great qual...


In [26]:
# Save data
df_reviews.to_csv('reviews.csv', index=False)